In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [99]:
from chatrag.csv_meta_loader import CSVMetaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from openai import OpenAI
import json

In [23]:
csv_path="../data/movies_title_overview_vote.csv"
metadata_columns_dtypes={"vote_average": "float"}
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [24]:
loader = CSVMetaLoader(csv_path, metadata_columns_dtypes=metadata_columns_dtypes)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=0)
index_creator = VectorstoreIndexCreator(text_splitter=text_splitter, embedding=embedding, vectorstore_cls=Chroma)

In [25]:
docsearch = index_creator.from_loaders([loader])

In [28]:
test = docsearch.vectorstore.similarity_search("What is the best movie ever?", k=4, filter={"vote_average": {"$gte": 9}})

In [33]:
print("\n\n".join([doc.page_content for doc in test]))


title: Stiff Upper Lips
overview: Stiff Upper Lips is a broad parody of British period films, especially the lavish Merchant-Ivory productions of the 'eighties and early 'nineties. Although it specifically targets A Room with a View, Chariots of Fire, Maurice, A Passage to India, and many other films, in a more general way Stiff Upper Lips satirises popular perceptions of certain Edwardian traits: propriety, sexual repression, xenophobia, and class snobbery.
vote_average: 10.0

title: Sardaarji
overview: A ghost hunter uses bottles to capture troublesome spirits.
vote_average: 9.5

title: Little Big Top
overview: An aging out of work clown returns to his small hometown, resigned to spend the rest of his days in a drunken stupor. But when his passion for clowning is reawakened by the local amateur circus he finds his smile.
vote_average: 10.0

title: One Man's Hero
overview: One Man's Hero tells the little-known story of the "St. Patrick's Battalion" or "San Patricios," a group of mostl

In [104]:
def search_movies(descripcion_pelicula: str, n_recomendaciones: int = 5, puntuacion_minima: float = 0) -> str:
    docs = docsearch.vectorstore.similarity_search(
        descripcion_pelicula, k=n_recomendaciones, filter={"vote_average": {"$gte": puntuacion_minima}}
    )
    return "\n\n".join([doc.page_content for doc in docs])

In [105]:
tool_name_dict = {"search_movies": search_movies}
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_movies",
            "description": "Search movies based on a description or synopsis and optionally a minimum rating.",
            "parameters": {
                "type": "object",
                "properties": {
                    "descripcion_pelicula": {
                        "type": "string",
                        "description": "Descripcion o sinopsis del tipo de película a buscar",
                    },
                    "puntuacion_minima": {
                        "type": "number",
                        "description": "Puntuación mínima de la película del 0 al 10.",
                    },
                    "n_recomendaciones": {
                        "type": "integer",
                        "description": "Número de peliculas recomendadas a recibir.",
                    },
                },
                "required": ["descripcion_pelicula"],
            },
        },
    }
]

In [45]:
oai_client = OpenAI()

In [167]:
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam
from openai.types.chat.chat_completion_message import ChatCompletionMessage

In [172]:
def chatbot(history: list[dict[str, str]|ChatCompletionMessageParam|ChatCompletionMessage]) -> str:
    response = oai_client.chat.completions.create(
        # model="gpt-4-turbo",
        model="gpt-3.5-turbo-0125",
        messages=history, #type: ignore
        tools=tools, #type: ignore
        tool_choice="auto",
    )
    choice = response.choices[0]
    if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
        tool_call = choice.message.tool_calls[0]
        function_name = tool_call.function.name
        arguments_json = tool_call.function.arguments
        # Convert the JSON string of arguments to a dictionary
        arguments_dict = json.loads(arguments_json)
        function = tool_name_dict[function_name]
        context = function(**arguments_dict)
        history = (
            history
            + [choice.message]
            + [{"role": "tool", "tool_call_id": tool_call.id, "content": context}]
        )
        print("vuelta")
        return chatbot(history)
    elif response.choices[0].message.content:
        return response.choices[0].message.content
    else:
        raise ValueError("Hay algo mal en el objeto response:\n", response)

In [173]:
system_prompt = """Eres un asistente de peliculas. Tu tarea es guiar al usuario para utilizar la película que mejor se ajuste a sus requisitos.

Usa la herramienta de busqueda de herramienta cuando hayas recogido suficiente información. Entonces, responde siguiendo estas reglas:
- Responde en el idioma del mensaje del usuario.
- Cuando tengas las propuestas de películas, evalua como de buena o mala propuesta es en una frase corta.
- Ofrece la opción de refinar la solicitud con nuevos requisitos si las sugerencias no son buenas.
- Si las sugerencias son buenas, simplemente pregunta al usuario si está satisfecho."""

historial_msjs = [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Hola, quiero ver una pelicula sobre vaqueros en el espacio."}]

In [174]:
test_out = chatbot(historial_msjs)

vuelta


In [176]:
print(test_out)

Te puedo ofrecer algunas películas relacionadas con vaqueros que podrían interesarte:

1. **Hidalgo** - Ambientada en 1890, narra la historia de un mensajero del Pony Express que viaja a Arabia para competir con su caballo, Hidalgo, en una carrera peligrosa alrededor del mundo. *(Puntuación: 6.5)*

2. **All the Pretty Horses** - Situada en 1949, sigue a un joven texano llamado John Grady que se encuentra sin hogar y se aventura al sur de la frontera en busca de una nueva vida como vaquero. *(Puntuación: 5.8)*

3. **Saving Private Perez** - Un hombre llamado Julián Pérez debe cumplir una peligrosa misión en Irak para salvar a su hermano soldado, comandando un grupo de élite reclutado en Sinaloa. *(Puntuación: 6.1)*

¿Alguna de estas películas es lo que estás buscando?
